# Invasive Species Monitoring

This notebook contains all work for the submission to the invasive species monitoring competion on Kaggle. The goal is first to see how far it is possible to get using a standard CNN. Aftervards pretrained models might be tested.

## Initial setup

We start by importing libraries and setting up helper functions.

In [24]:
import numpy as np
import pandas as pd
import os
import sys

from shutil import move

from keras.preprocessing import image

Set global variables

In [6]:
# Define home directory variable
current_dir = os.getcwd()
DATA_HOME_DIR = current_dir
path = 'sample/'

# Allow relative imports to directories above main
sys.path.insert(1, os.path.join(sys.path[0], '..'))

Define helper functions

In [9]:
def get_batches(path, gen = image.ImageDataGenerator(), shuffle = True,
                batch_size = 8, class_mode = 'categorical',
                target_size = (480, 640)):
    
    return gen.flow_from_directory(path, target_size = target_size,
                                   class_mode = class_mode, shuffle = shuffle,
                                   batch_size = batch_size)

## Setup data

We create the right folder structure for our data.

Start by making the folders currently missing.

In [12]:
%mkdir -p train/0
%mkdir -p train/1
%mkdir -p valid/0
%mkdir -p valid/1
%mkdir -p test/unknown
%mkdir models
%mkdir results

We need to identify the training images of each category, by reading from a .csv file.

In [23]:
files = pd.read_csv('train_labels.csv')

files['name'] = files['name'].apply(lambda x: str(x) + '.jpg')

invasives = files[files['invasive'] == 1]['name'].tolist()
not_invasives = files[files['invasive'] == 0]['name'].tolist()


Then we move each file to their respective folders.

In [31]:
for f in invasives:
    move('train/' + f, 'train/1')

In [32]:
for f in not_invasives:
    move('train/' + f, 'train/0')

We then move/copy files to validation and sample folders respectively.

In [33]:
%cd $DATA_HOME_DIR/train/0
!shuf -n 170 -e * | xargs -i mv {} ~/kaggle/invasive-species-monitoring/valid/0

%cd $DATA_HOME_DIR/train/1
!shuf -n 290 -e * | xargs -i mv {} ~/kaggle/invasive-species-monitoring/valid/1

%cd $DATA_HOME_DIR

/home/ubuntu/kaggle/invasive-species-monitoring/train/0
/home/ubuntu/kaggle/invasive-species-monitoring/train/1
/home/ubuntu/kaggle/invasive-species-monitoring


And we're all set.

## Simple linear model

We start out by using a simple linear model so as to get a baseline prediction.